<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Image
### Method: CLIP + Siren
This model combines an image classifier called **CLIP** from *OpenAI* and an image generator called **SIREN.**

The process works like this:
1.  The **SIREN** model at first generates a completely random image
2.  The **CLIP** image classifier looks at the image and compares it with the given text caption
3.  Using this feedback the weights of the **SIREN** model are adjusted slightly to better approximate the image caption
4.  These steps are repeated until we stop it manually
---
In order to run click the [Colab] link. And then select *Runtime -> Run all* from the menu bar.

[colab]: https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb

## Parameters

In [1]:
text = 'An illustration of a black and white cat'  #@param {type: "string"}
args = '--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123'  #@param {type: "string"}

parameters = {
    "text": text,
    "args": args
}

## Code (Frontend and IPFS Connection)

In [3]:
#@title Empty and create ipfs root: */content/ipfs*

ipfs_root = "/content/ipfs"

!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!echo "Created IPFS resource" > $ipfs_root/output/log

for key, default in parameters.items():
  print(default, file=open(f"{ipfs_root}/input/{key}", 'a'))



In [4]:
#@title Install IPFS-GO

!pip install aioipfs nest-asyncio

!wget wget https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
!tar -xvzf go-ipfs_v0.8.0_linux-amd64.tar.gz
%cd go-ipfs
!sudo bash install.sh
%cd -

#!npm install -g nodemon


--2021-05-02 10:53:51--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2021-05-02 10:53:51--  https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
Resolving dist.ipfs.io (dist.ipfs.io)... 209.94.78.1, 2602:fea2:3::1
Connecting to dist.ipfs.io (dist.ipfs.io)|209.94.78.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25904633 (25M) [application/gzip]
Saving to: ‘go-ipfs_v0.8.0_linux-amd64.tar.gz.1’

go-ipfs_v0.8.0_linu 100%[===================>]  24.70M  --.-KB/s    in 0.1s    

2021-05-02 10:53:51 (239 MB/s) - ‘go-ipfs_v0.8.0_linux-amd64.tar.gz.1’ saved [25904633/25904633]

FINISHED --2021-05-02 10:53:51--
Total wall clock time: 0.2s
Downloaded: 1 files, 25M in 0.1s (239 MB/s)
go-ipfs/install.sh
go-ipfs/ipfs
go-ipfs/LICENSE
go-ipfs/LICENSE-APACHE
go-ipfs/LICENSE-MIT
go-ipfs/README.md
/content/go-ipfs
Moved ./ipfs to /usr/local/bin
/content


In [5]:
#@title Start IPFS daemon and connect to public daemon
!killall ipfs 2>/dev/null
!sleep 2
!killall -9 ipfs 2>/dev/null
IPFS_PEER="18.157.173.110"
#!nohup jsipfs daemon --enable-namesys-pubsub &
!ipfs init
!ipfs config Addresses.Gateway /ip4/0.0.0.0/tcp/9001
!nohup ipfs daemon --enable-namesys-pubsub --enable-pubsub-experiment &
!sleep 12
#!ipfs config Addresses.API /ip4/127.0.0.1/tcp/5002
#!jsipfs config --json API.HTTPHeaders.Access-Control-Allow-Origin '["*"]'
#!jsipfs config --json API.HTTPHeaders.Access-Control-Allow-Methods '["PUT", "GET", "POST"]'
!tail nohup.out
!ipfs swarm connect /ip4/$IPFS_PEER/tcp/4002/p2p/QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr


generating ED25519 keypair...done
peer identity: 12D3KooWPnwDVQL9SDTHGUVfjGGEsBNa4D28ttsX9iiKYTqVUBos
initializing IPFS node at /root/.ipfs
Error: ipfs configuration file already exists!
Reinitializing would overwrite your keys.

nohup: appending output to 'nohup.out'
Swarm listening on /ip4/172.28.0.2/udp/4001/quic
Swarm listening on /p2p-circuit
Swarm announcing /ip4/127.0.0.1/tcp/4001
Swarm announcing /ip4/127.0.0.1/udp/4001/quic
Swarm announcing /ip4/172.28.0.2/tcp/4001
Swarm announcing /ip4/172.28.0.2/udp/4001/quic
API server listening on /ip4/127.0.0.1/tcp/5001
WebUI: http://127.0.0.1:5001/webui
Gateway (readonly) server listening on /ip4/0.0.0.0/tcp/9001
Daemon is ready
connect QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr success


In [8]:
#@title Connect to IPFS Pubsub

async def sub(topic, handler=None):
  print("subscribed to",topic)
  async for message in client.pubsub.sub(topic):
    if handler:
      handler(message)
    else:
      print("No handler. Received IPFS pub message",message)

async def pub(topic, data):
  await client.pubsub.pub(topic, data)

def on_frontend_data(handler):
  def handle(ipfs_message):
    handler(ipfs_message["data"])
  asyncio.run(loop.create_task(sub(ipfs_nodeid, handle)))

def update_frontend():
  asyncio.run(loop.create_task(pub(ipfs_nodeid, ipfs_contentid)))


In [9]:
#@title Connect to local IPFS node and publish ipfs root

import sys
import asyncio
import aioipfs
import nest_asyncio
import json

client = aioipfs.AsyncIPFS(debug=True)
clientRemote = aioipfs.AsyncIPFS(host=IPFS_PEER,port=5002,debug=True)

ipfs_nodeid = None

async def get_node_id():
    return (await client.id())["ID"]

async def add_and_publish(path=ipfs_root, pin_remote=False):
    last_added=None
    async for added_file in client.add(path, recursive=True):
      print('Imported file', added_file)
      last_added=added_file["Hash"]
    
    if pin_remote:
      async for added_pin in clientRemote.pin.add(last_added):
        print("Added pin", added_pin)
    
    return last_added

loop = asyncio.get_running_loop()
nest_asyncio.apply(loop)

ipfs_contentid = asyncio.run(loop.create_task(add_and_publish(pin_remote=True)))
ipfs_nodeid = asyncio.run(loop.create_task(get_node_id()))



#asyncio.run(loop.create_task(sub("hello")))
print("Got node id:",ipfs_nodeid," content id:",ipfs_contentid)

update_frontend()

Imported file {'Name': 'ipfs/input/args', 'Hash': 'QmThTzcq3VshSiK3y4ytdJ1WXzXxjis3cjDUayDtCtqMfm', 'Size': '185'}
Imported file {'Name': 'ipfs/input/text', 'Hash': 'Qma6UpYxGUCKJcrhhJ18KBsg5Je11X2JnFjCKoqVduFBGX', 'Size': '49'}
Imported file {'Name': 'ipfs/output/log', 'Hash': 'Qmbj3AxwdsTAtgDDEt91Yr7RaHqBTseABrnKyFzNycjjuH', 'Size': '30'}
Imported file {'Name': 'ipfs/input', 'Hash': 'QmXcYVE6HBYmJizYPYWvVDqhSXFD718gn2UkFXUmVY6zeF', 'Size': '331'}
Imported file {'Name': 'ipfs/output', 'Hash': 'QmU5jczrSj1WhiYE95E7KXyPq46oqY7yMmSgM6d4en74va', 'Size': '79'}
Imported file {'Name': 'ipfs', 'Hash': 'QmZ6Yo8bps8HisxoNTCkxUwiimi2nw9zYnDWftjfW51JHL', 'Size': '510'}
Added pin {'Pins': ['QmZ6Yo8bps8HisxoNTCkxUwiimi2nw9zYnDWftjfW51JHL']}
Got node id: 12D3KooWBGCfvngY4bdtECvJHm36yx5XrzPyeE3MeR242AcisFZT  content id: QmZ6Yo8bps8HisxoNTCkxUwiimi2nw9zYnDWftjfW51JHL


In [10]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `http://localhost:3000/incolab.html?node={ipfs_nodeid}`

  document.querySelector("#output-area").appendChild(iframe)
'''))



<IPython.core.display.Javascript object>

In [ ]:
on_frontend_data(print)

subscribed to 12D3KooWBGCfvngY4bdtECvJHm36yx5XrzPyeE3MeR242AcisFZT


## CODE (DEEP-DAZE)

In [ ]:
!pip install deep-daze --upgrade
!pip install imageio-ffmpeg asyncio websockets varname --upgrade

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 
import re
flatten = itertools.chain.from_iterable

TEXT="A black and white technical illustration of merging a cat and a human through biohacking."
DEFAULT_PARAMS="--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=1234"
GITHUB_REPO="voodoohop/pollinations/blob/master/colabs/deep-daze.ipynb"
GITHUB_REPO_CLEAN = re.sub('[^0-9a-zA-Z]+', '_', GITHUB_REPO)
print(GITHUB_REPO_CLEAN)

def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  print("Running",text)
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  log_out = open(f"{ipfs_root}/colabConsole.txt","a")
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    log_out.write(line.decode("utf-8"))
    log_out.flush()
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish(pin)          

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))


#conn.set_listener('', MyListener(conn,run))
print(sub)
#run(TEXT)
#conn.disconnect()
sleep(1000000)


In [ ]:
conn.disconnect()